In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import display, Markdown, update_display
from openai import OpenAI


In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

model = "gpt-4o-mini"
openai = OpenAI()

In [3]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    url: str
    title: str
    body: str
    links: List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
print(ed.get_contents())

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers a

In [5]:
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON. If I provide you with a list of links such as the one below:\n"
link_system_prompt += """
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/
https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
mailto:hello@mygroovydomain.com
https://www.linkedin.com/in/eddonner/
https://twitter.com/edwarddonner
https://www.facebook.com/edward.donner.52
"""

link_system_prompt += "You should respond with a JSON object that looks like this:\n"

link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://edwarddonner.com/about-me-and-about-nebula/"},
        {"type": "posts page", "url": "https://edwarddonner.com/posts/"},
        {"type": "find me on linkedin", "url": "https://www.linkedin.com/in/eddonner/"}
        ]
}
"""

link_system_prompt += "And another example:\n"
link_system_prompt += "If I provide you with a list of links such as the one below:\n"

link_system_prompt += """
 'https://www.anthropic.com/',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://www.anthropic.com/pricing',
 'https://claude.ai/download',
 'https://claude.ai/',
 'https://www.anthropic.com/news/claude-character',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/pricing#api',
 'https://console.anthropic.com/',
 'https://docs.anthropic.com/en/docs/welcome',
 'https://www.anthropic.com/solutions/agents',
 'https://www.anthropic.com/solutions/coding',
 'https://www.anthropic.com/solutions/customer-support',
 'https://www.anthropic.com/solutions/education',
 'https://www.anthropic.com/solutions/financial-services',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/research',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/claude/opus',
 'https://www.anthropic.com/claude/sonnet',
 'https://www.anthropic.com/claude/haiku',
 'https://www.anthropic.com/news/visible-extended-thinking',
 'https://www.anthropic.com/transparency',
 'https://www.anthropic.com/news/announcing-our-updated-responsible-scaling-policy',
 'https://www.anthropic.com/claude/sonnet',
 'https://www.anthropic.com/news/anthropic-achieves-iso-42001-certification-for-responsible-ai',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/engineering',
 'https://www.anthropic.com/learn',
 'https://www.anthropic.com/company',
 'https://www.anthropic.com/careers',
 'https://www.anthropic.com/events',
 'https://www.anthropic.com/engineering/building-effective-agents',
 'https://www.anthropic.com/news',
 '#',
 'https://claude.ai/login',
 'https://claude.ai/download',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://www.anthropic.com/pricing',
 'https://claude.ai/download',
 'https://claude.ai/',
 'https://www.anthropic.com/news/claude-character',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/pricing#api',
 'https://console.anthropic.com/',
 'https://docs.anthropic.com/en/docs/welcome',
 'https://www.anthropic.com/solutions/agents',
 'https://www.anthropic.com/solutions/coding',
 'https://www.anthropic.com/solutions/customer-support',
 'https://www.anthropic.com/solutions/education',
 'https://www.anthropic.com/solutions/financial-services',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/research',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/claude/opus',
 'https://www.anthropic.com/claude/sonnet',
 'https://www.anthropic.com/claude/haiku',
 'https://www.anthropic.com/news/visible-extended-thinking',
 'https://www.anthropic.com/transparency',
 'https://www.anthropic.com/news/announcing-our-updated-responsible-scaling-policy',
 'https://www.anthropic.com/claude/sonnet',
 'https://www.anthropic.com/news/anthropic-achieves-iso-42001-certification-for-responsible-ai',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/engineering',
 'https://www.anthropic.com/learn',
 'https://www.anthropic.com/company',
 'https://www.anthropic.com/careers',
 'https://www.anthropic.com/events',
 'https://www.anthropic.com/engineering/building-effective-agents',
 'https://www.anthropic.com/news',
 '#',
 'https://claude.ai/login',
 'https://claude.ai/download',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 'https://www.anthropic.com/api',
 'https://anthropic.com/news/claude-4',
 'https://www.anthropic.com/claude/opus',
 'https://www.anthropic.com/claude/sonnet',
 'https://www.anthropic.com/news/core-views-on-ai-safety',
 'https://www.anthropic.com/rsp-updates',
 'https://www.anthropic.com/learn',
 'https://www.anthropic.com/research/project-vend-1',
 'https://www.anthropic.com/research/agentic-misalignment',
 'https://www.anthropic.com/news/claude-4',
 'https://www.anthropic.com/research/tracing-thoughts-language-model',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/news/model-context-protocol',
 'https://www.anthropic.com/jobs',
 'https://www.anthropic.com/jobs',
 'https://www.anthropic.com/contact-sales',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/claude-code',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://claude.ai/download',
 'https://www.anthropic.com/pricing',
 'http://claude.ai/login',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/amazon-bedrock',
 'https://www.anthropic.com/google-cloud-vertex-ai',
 'https://www.anthropic.com/pricing#anthropic-api'
 """

link_system_prompt += "You should respond with a JSON object that looks like this:\n"

link_system_prompt += """
{
    "links": [
        {"type": "our team", "url": "https://www.anthropic.com/team"},
        {"type": "events page", "url": "https://www.anthropic.com/events"},
        {"type": "about company page", "url": "https://www.anthropic.com/company"},
        {"type": "careers page", "url": "https://www.anthropic.com/careers"},
        {"type": "contact sales page", "url": "https://www.anthropic.com/contact-sales"},
        {"type": "claude 4", "url": "https://www.anthropic.com/news/claude-4"}
    ]
}
"""

In [7]:
print(link_system_prompt) 

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON. If I provide you with a list of links such as the one below:

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-ex

In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [10]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=model,
        messages = [
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [11]:
get_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'posts page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'find me on linkedin',
   'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'connect four', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'outsmart', 'url': 'https://edwarddonner.com/outsmart/'}]}

In [12]:
get_links("https://anthropic.com")

{'links': [{'type': 'company page',
   'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'events page', 'url': 'https://www.anthropic.com/events'},
  {'type': 'our team', 'url': 'https://www.anthropic.com/team'},
  {'type': 'engineering page', 'url': 'https://www.anthropic.com/engineering'},
  {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'},
  {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'},
  {'type': 'solutions page',
   'url': 'https://www.anthropic.com/solutions/agents'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'},
  {'type': 'API page', 'url': 'https://www.anthropic.com/api'},
  {'type': 'pricings page', 'url': 'https://www.anthropic.com/pricing'},
  {'type': 'contact sales page',
   'url': 'https://www.anthropic.com/contact-sales'},
  {'type': 'transparency page',
   'url': 'https://www.anthropic.com/transparency'},
  {'type': 'status page'

In [13]:
anthropic = Website("https://anthropic.com")
anthropic.links

['#main',
 '#footer',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://www.anthropic.com/pricing',
 'https://claude.ai/download',
 'https://claude.ai/',
 'https://www.anthropic.com/news/claude-character',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/pricing#api',
 'https://console.anthropic.com/',
 'https://docs.anthropic.com/en/docs/welcome',
 'https://www.anthropic.com/solutions/agents',
 'https://www.anthropic.com/solutions/coding',
 'https://www.anthropic.com/solutions/customer-support',
 'https://www.anthropic.com/solutions/education',
 'https://www.anthropic.com/solutions/financial-services',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/research',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/claude/opus',
 'https://www.anthropic.com/claude/sonne

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links", links)
    for link in links['links']:
        result += f"\n\n{link['type']}\n"
        result += Website(link['url']).get_contents()
    return result

print(get_all_details("https://anthropic.com"))

Found links {'links': [{'type': 'homepage', 'url': 'https://www.anthropic.com/'}, {'type': 'about company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'our team', 'url': 'https://www.anthropic.com/team'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'engineering page', 'url': 'https://www.anthropic.com/engineering'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions/coding'}, {'type': 'api page', 'url': 'https://www.anthropic.com/api'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}, {'type': 'contact sales page', 'url': 'https://www.anthropic.com/contact-sales'}, {'type': 'news page', 'url': 'https://www.anth

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here iare the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt += user_prompt[:20_000]
    return user_prompt

In [17]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Found links {'links': [{'type': 'homepage', 'url': 'https://www.anthropic.com/'}, {'type': 'our team', 'url': 'https://www.anthropic.com/team'}, {'type': 'about company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'engineering page', 'url': 'https://www.anthropic.com/engineering'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}, {'type': 'enterprise solutions', 'url': 'https://www.anthropic.com/enterprise'}, {'type': 'solutions - agents', 'url': 'https://www.anthropic.com/solutions/agents'}, {'type': 'solutions - coding', 'url': 'https://www.anthropic.com/solutions/coding'}, {'type': 'solutions - customer support', 'url': 'https://www.anthropic.com/solutions/custome

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


'You are looking at a company called: Anthropic\nHere iare the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nChat with Claude\nOverview\nMax plan\nTeam plan\nEnterprise plan\nExplore pricing\nDownload apps\nClaude log in\nNews\nClaude’s character\nAPI\nBuild with Claude\nAPI\xa0overview\nDeveloper docs\nExplore pricing\nConsole log in\nNews\nLearn how to build with Claude\nSolutions\nCollaborate with Claude\nAI\xa0agents\nCoding\nCustomer support\nEducation\nFinancial services\nCase studies\nHear from our customers\nResearch\nResearch\nOverview\nEconomic Index\nClaude model family\nClaude Opus 4\nClaude Sonnet 4\nClaude Haiku 3.5\nResearch\nClaude’s extended thinking\nCommitments\nInitiatives\nTransparency\nResponsible scaling policy\nTrust center\nSecurity and compliance\nAnnouncement\nIS

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ]
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("Anthropic", "https://anthropic.com")

Found links {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'our team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'engineering page', 'url': 'https://www.anthropic.com/engineering'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions/coding'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'pricing page', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}]}


```markdown
# Anthropic Company Brochure

## About Us

**Anthropic** is an AI safety and research company based in San Francisco dedicated to building reliable, interpretable, and steerable AI systems. Our focus is on ensuring that artificial intelligence serves humanity's long-term well-being through responsible development.

### Our Purpose

We believe AI will have a vast impact on the world. That’s why we build tools with human benefit at their foundation. Our flagship product, **Claude**, encapsulates this mission by offering advanced AI capabilities to various sectors.

### Core Values

1. **Act for the Global Good**: We strive to maximize positive outcomes for humanity and take bold actions to ensure technology is a force for good.
2. **Hold Light and Shade**: Acknowledge the potential risks and benefits of AI.
3. **Be Good to Our Users**: We define users broadly, ensuring kindness and generosity in our interactions.
4. **Ignite a Race to the Top on Safety**: Safety in AI systems is a collective responsibility, and we aim to inspire excellence within the industry.
5. **Do the Simple Thing That Works**: We focus on empirical evidence and practical solutions.
6. **Be Helpful, Honest, and Harmless**: Create a low-ego environment where everyone contributes towards collective goals.
7. **Put the Mission First**: Our mission provides a shared purpose that guides our decisions and actions.

## Our Products

### Claude
Meet **Claude**, our suite of AI models designed for various applications:
- **Claude Opus 4**: The latest and most advanced model, pioneering capabilities for coding and AI agents.
- **Claude Sonnet 4**: Optimized for high performance, perfect for developers.
- **Claude Haiku 3.5**: Focused on speed and cost-effectiveness.

### Solutions and Applications

Our AI systems can be applied across industries:
- **Education**: Personalized learning experiences.
- **Financial Services**: Advanced decision-making tools.
- **Customer Support**: Enhance user experience and operational efficiency.

## Customer Impact

Leading enterprises and startups utilize Claude to transform their operations:
- **Gradient Labs**: Revolutionized customer support in financial services.
- **StubHub**: Streamlined ticketing processes.
- **Zapier**: Built an AI-first company culture.

## Join Us

### Careers

At Anthropic, we prioritize building diverse teams with backgrounds in physics, machine learning, public policy, and business. We offer:
- **Comprehensive Health Benefits**: Including dental and vision coverage.
- **Flexible Paid Time Off**: Supporting work-life balance.
- **Educational Support**: Stipends for continuous learning and skill development.

### Current Open Roles

We are continuously looking for talented individuals across technical and operational roles. **Your unique skill set might be just what we need to help shape the future of safe AI!**

## Vision for the Future

We view ourselves as one piece of a larger puzzle in the ongoing evolution of AI. Collaborating with civil society, government, and academia is crucial for a safe AI ecosystem.

---

For more information, visit our website at [Anthropic](https://www.anthropic.com).
```


In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )

    response = ""

    display_handle = display(Markdown(""),  display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

    return response

In [21]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links {'links': [{'type': 'homepage', 'url': 'https://www.anthropic.com/'}, {'type': 'our team', 'url': 'https://www.anthropic.com/team'}, {'type': 'pricing page', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'about company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'learn more', 'url': 'https://www.anthropic.com/learn'}, {'type': 'contact sales page', 'url': 'https://www.anthropic.com/contact-sales'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions/agents'}, {'type': 'engineering page', 'url': 'https://www.anthropic.com/engineering'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}]}


# Anthropic Brochure

## About Anthropic

**Anthropic** is an AI safety and research company focused on creating reliable, interpretable, and steerable AI systems to serve humanity's long-term well-being. Headquartered in San Francisco, Anthropic operates as a Public Benefit Corporation, prioritizing global good in AI development while promoting safety and accountability. 

### Mission
Our mission is to advance AI in ways that are beneficial for all, recognizing that while powerful technologies can bring significant benefits, they also require careful consideration of their societal impacts.

### Core Values
1. **Act for the Global Good**: We make decisions that maximize positive outcomes for humanity.
2. **Hold Light and Shade**: We balance the potential risks and rewards that AI presents.
3. **Be Good to Our Users**: We cultivate kindness and generosity in interactions with all stakeholders.
4. **Ignite a Race to the Top on Safety**: We strive to set high industry standards for AI safety and security.
5. **Do the Simple Thing That Works**: We focus on empirical approaches that yield significant impacts.
6. **Be Helpful, Honest, and Harmless**: We promise clear communication and collaborative problem-solving.
7. **Put the Mission First**: Our shared purpose guides our actions and decisions.

## Products and Solutions

### Meet Claude
**Claude** is Anthropic's AI assistant, designed to assist users in various domains, enhancing productivity and transforming workflows. Claude supports use cases in:

- **Coding**: Generate code snippets and debug effectively.
- **Customer Support**: Craft personalized, immediate responses to customer inquiries.
- **Education**: Facilitate engaging, interactive learning experiences.
- **Marketing & Sales**: Create targeted, compelling content and automated outreach.

### Claude Models
- **Claude Opus 4**: The most intelligent model, optimized for reasoning and task execution.
- **Claude Sonnet 4 & Haiku 3.5**: Tailored for a range of applications, from complex coding to conversational tasks.

### API and Integration
Developers can build applications integrated with Claude using our powerful API, enhancing performance across various platforms, including Google Cloud and Amazon Web Services.

## Company Culture

At Anthropic, our culture is underpinned by collaboration, inclusivity, and a commitment to rigorous research practices. We combine expertise from various fields including AI research, engineering, policy-making, and business to ensure a multi-faceted approach to AI development. 

### Work Environment
We foster an open and supportive work culture where employees are encouraged to bring diverse perspectives. Benefits include:
- Comprehensive health plans
- Flexible time off
- Generous parental leave
- Professional development opportunities 
- A wellness stipend to support work-life balance

## Careers at Anthropic

We are always looking for passionate individuals eager to contribute to the safe development of AI. An interdisciplinary team allows us to innovate and adapt quickly to the evolving demands of AI technology.

### Current Openings
Visit our careers page to explore available roles, ranging from technical positions to operational roles. We welcome applicants from all backgrounds, whether or not they have prior machine learning experience.

- Join us in shaping the future of safe AI!
- See open roles on our [Careers Page](https://www.anthropic.com/careers).

## Get in Touch

To learn more about Anthropic or to get assistance:
- [Contact us](https://www.anthropic.com/contact) for inquiries
- Visit our [Solutions Page](https://www.anthropic.com/solutions) for detailed insights into our offerings and integrations.

### Follow us
Stay updated with the latest from Anthropic through our social channels and newsroom.

## Conclusion

With a commitment to safety, transparency, and the promotion of human welfare, Anthropic is at the forefront of responsible AI development. Join us as we pave the way for future advancements in artificial intelligence!

"# Anthropic Brochure\n\n## About Anthropic\n\n**Anthropic** is an AI safety and research company focused on creating reliable, interpretable, and steerable AI systems to serve humanity's long-term well-being. Headquartered in San Francisco, Anthropic operates as a Public Benefit Corporation, prioritizing global good in AI development while promoting safety and accountability. \n\n### Mission\nOur mission is to advance AI in ways that are beneficial for all, recognizing that while powerful technologies can bring significant benefits, they also require careful consideration of their societal impacts.\n\n### Core Values\n1. **Act for the Global Good**: We make decisions that maximize positive outcomes for humanity.\n2. **Hold Light and Shade**: We balance the potential risks and rewards that AI presents.\n3. **Be Good to Our Users**: We cultivate kindness and generosity in interactions with all stakeholders.\n4. **Ignite a Race to the Top on Safety**: We strive to set high industry stand

In [24]:
hg_brochure = stream_brochure("HuggingFace", "https://huggingface.co")

Found links {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}]}


# Hugging Face Brochure

## **About Us**
Hugging Face is at the forefront of the artificial intelligence revolution, fostering a vibrant community that is dedicated to building the future of AI. We offer a collaborative platform for machine learning enthusiasts, researchers, and companies to share models, datasets, and applications. With over **1 million models** and **250,000 datasets** available, Hugging Face is the hub for AI collaboration.

## **Our Offerings**
### **Models and Datasets**
- Access a comprehensive range of **1M+ models** for diverse tasks such as text generation, image processing, and speech synthesis.
- Utilize **250k+ datasets** tailored for various machine learning tasks, from multimodal tasks to medical imaging and more.

### **Spaces**
- Explore **AI Spaces**, where users can build and share applications across various domains, including:
  - Image Generation
  - Language Translation
  - Video Generation
  - Code Generation

### **Enterprise Solutions**
- Our enterprise solutions include tiered packages that provide organizations with the resources and tools to build robust AI applications. Plans start at **$20/user/month** and include:
  - Dedicated support
  - Single Sign-On for secure access
  - Advanced compute options

## **Community and Culture**
Our community-driven approach allows us to cultivate a culture of innovation, creativity, and collaboration. With **over 50,000 organizations** leveraging our platform (including giants like Google, Amazon, and Microsoft), we actively contribute to making AI accessible and democratized.

### **Join Us**
We're looking for passionate individuals to join our team. We value diversity and foster an inclusive culture where every voice matters. Whether you’re a seasoned professional or just starting, there’s a place for you at Hugging Face.

## **Careers at Hugging Face**
Explore our diverse career opportunities in fields like:
- Machine Learning Engineering
- Data Science
- Software Engineering
- Community Management

Our team is committed to continuous learning, where we support each other in professional development through resources like courses and community events.

## **Contact Us**
Join the revolution and build the future with us!  
Explore opportunities or get started with Hugging Face today.  
Visit us at: [huggingface.co](https://huggingface.co)

### **Follow Us**
Stay up-to-date with the latest in AI and Hugging Face by following us on social media:  
- [GitHub](https://github.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)
- [Twitter](https://twitter.com/huggingface)

---

**Hugging Face**: The AI community building the future.  
Together, let's unlock new possibilities with machine learning!

In [23]:
anthropic.links

['#main',
 '#footer',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://www.anthropic.com/pricing',
 'https://claude.ai/download',
 'https://claude.ai/',
 'https://www.anthropic.com/news/claude-character',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/pricing#api',
 'https://console.anthropic.com/',
 'https://docs.anthropic.com/en/docs/welcome',
 'https://www.anthropic.com/solutions/agents',
 'https://www.anthropic.com/solutions/coding',
 'https://www.anthropic.com/solutions/customer-support',
 'https://www.anthropic.com/solutions/education',
 'https://www.anthropic.com/solutions/financial-services',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/research',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/claude/opus',
 'https://www.anthropic.com/claude/sonne

In [25]:
translation_system_prompt = "You are provided with a brochure about a company in English. Your task is to translate it into Spanish without changing the meaning or the tone of the original text."

def get_translation(text):
    stream = openai.chat.completions.create(
        model=model,
        messages = [
            {"role": "system", "content": translation_system_prompt},
            {"role": "user", "content": text}
        ],
        stream=True
    )
    response = ""

    display_handle = display(Markdown(""),  display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
    return response

In [26]:
get_translation(text=hg_brochure)

# Folleto de Hugging Face

## **Sobre Nosotros**
Hugging Face está a la vanguardia de la revolución de la inteligencia artificial, fomentando una comunidad vibrante dedicada a construir el futuro de la IA. Ofrecemos una plataforma colaborativa para entusiastas del aprendizaje automático, investigadores y empresas para compartir modelos, conjuntos de datos y aplicaciones. Con más de **1 millón de modelos** y **250,000 conjuntos de datos** disponibles, Hugging Face es el centro de colaboración en IA.

## **Nuestra Oferta**
### **Modelos y Conjuntos de Datos**
- Accede a una amplia gama de **más de 1M modelos** para diversas tareas como generación de texto, procesamiento de imágenes y síntesis de voz.
- Utiliza **más de 250k conjuntos de datos** diseñados para diversas tareas de aprendizaje automático, desde tareas multimodales hasta imágenes médicas y más.

### **Espacios**
- Explora los **Espacios de IA**, donde los usuarios pueden construir y compartir aplicaciones en varios dominios, incluyendo:
  - Generación de Imágenes
  - Traducción de Idiomas
  - Generación de Video
  - Generación de Código

### **Soluciones Empresariales**
- Nuestras soluciones empresariales incluyen paquetes escalonados que proporcionan a las organizaciones los recursos y herramientas para construir aplicaciones robustas de IA. Los planes comienzan en **$20/usuario/mes** e incluyen:
  - Soporte dedicado
  - Inicio de sesión único para acceso seguro
  - Opciones de cómputo avanzadas

## **Comunidad y Cultura**
Nuestro enfoque impulsado por la comunidad nos permite cultivar una cultura de innovación, creatividad y colaboración. Con **más de 50,000 organizaciones** aprovechando nuestra plataforma (incluyendo gigantes como Google, Amazon y Microsoft), contribuimos activamente a hacer la IA accesible y democratizada.

### **Únete a Nosotros**
Buscamos individuos apasionados para unirse a nuestro equipo. Valoramos la diversidad y fomentamos una cultura inclusiva donde cada voz importa. Ya seas un profesional experimentado o estés comenzando, hay un lugar para ti en Hugging Face.

## **Carreras en Hugging Face**
Explora nuestras diversas oportunidades laborales en campos como:
- Ingeniería de Aprendizaje Automático
- Ciencia de Datos
- Ingeniería de Software
- Gestión de Comunidad

Nuestro equipo está comprometido con el aprendizaje continuo, donde nos apoyamos mutuamente en el desarrollo profesional a través de recursos como cursos y eventos comunitarios.

## **Contáctanos**
¡Únete a la revolución y construye el futuro con nosotros!  
Explora oportunidades o comience con Hugging Face hoy.  
Visítanos en: [huggingface.co](https://huggingface.co)

### **Síguenos**
Mantente al día con las últimas novedades en IA y Hugging Face siguiéndonos en las redes sociales:  
- [GitHub](https://github.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)
- [Twitter](https://twitter.com/huggingface)

---

**Hugging Face**: La comunidad de IA que construye el futuro.  
¡Juntos, desbloqueemos nuevas posibilidades con el aprendizaje automático!

'# Folleto de Hugging Face\n\n## **Sobre Nosotros**\nHugging Face está a la vanguardia de la revolución de la inteligencia artificial, fomentando una comunidad vibrante dedicada a construir el futuro de la IA. Ofrecemos una plataforma colaborativa para entusiastas del aprendizaje automático, investigadores y empresas para compartir modelos, conjuntos de datos y aplicaciones. Con más de **1 millón de modelos** y **250,000 conjuntos de datos** disponibles, Hugging Face es el centro de colaboración en IA.\n\n## **Nuestra Oferta**\n### **Modelos y Conjuntos de Datos**\n- Accede a una amplia gama de **más de 1M modelos** para diversas tareas como generación de texto, procesamiento de imágenes y síntesis de voz.\n- Utiliza **más de 250k conjuntos de datos** diseñados para diversas tareas de aprendizaje automático, desde tareas multimodales hasta imágenes médicas y más.\n\n### **Espacios**\n- Explora los **Espacios de IA**, donde los usuarios pueden construir y compartir aplicaciones en vario

In [2]:
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown, update_display
from openai import OpenAI



MODEL_GPT = "gpt-4o-mini"
MODEL_LLAMA = 'llama3.2'

system_prompt = "You are a coding expert capable of explaining code in a clear and concise manner."

question = """
Please explain wht this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai = OpenAI()

messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ]


def get_answer(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
    
    return response


get_answer(question)

The code you've provided is a Python expression that utilizes a few concepts:

1. **Set Comprehension**: The part `{book.get("author") for book in books if book.get("author")}` is a set comprehension. It creates a set of unique authors from a collection named `books`.

2. **`get` Method**: The `book.get("author")` method is used to extract the value associated with the key `"author"` from each `book` dictionary. If the key is not present, this method returns `None` (or a specified default value if provided). 

3. **Filtering**: The `if book.get("author")` condition filters out any `book` entries that do not have an author (i.e., where the author is `None` or does not exist).

4. **`yield from`**: The `yield from` statement is used in a generator function to yield all values from an iterable. In this context, it yields each author from the set created in the comprehension.

### Summary of What the Code Does
- It creates a set of unique authors from the `books` collection (which is expected to be a list of dictionaries, each representing a book).
- It filters out any books that do not have an author.
- It yields each unique author one at a time, allowing them to be consumed in a generator-like fashion.

### Example in Context
Here's an example to illustrate:

python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": None},  # No author provided
    {"title": "Book 4", "author": "Author A"}  # Duplicate author
]

def get_unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Using the generator
for author in get_unique_authors(books):
    print(author)


### Output

Author A
Author B


In this example:
- The function `get_unique_authors` yields unique authors from the `books` list.
- The set comprehension ensures duplicates are removed and only valid authors are included.

'The code you\'ve provided is a Python expression that utilizes a few concepts:\n\n1. **Set Comprehension**: The part `{book.get("author") for book in books if book.get("author")}` is a set comprehension. It creates a set of unique authors from a collection named `books`.\n\n2. **`get` Method**: The `book.get("author")` method is used to extract the value associated with the key `"author"` from each `book` dictionary. If the key is not present, this method returns `None` (or a specified default value if provided). \n\n3. **Filtering**: The `if book.get("author")` condition filters out any `book` entries that do not have an author (i.e., where the author is `None` or does not exist).\n\n4. **`yield from`**: The `yield from` statement is used in a generator function to yield all values from an iterable. In this context, it yields each author from the set created in the comprehension.\n\n### Summary of What the Code Does\n- It creates a set of unique authors from the `books` collection (w

In [6]:
import ollama

response = ollama.chat(model=MODEL_LLAMA, messages=messages, stream=True)
for i in response:
    print(i['message']['content'], end='', flush=True)

**Understanding the Code**

This line of code is written in Python and uses a feature called **generator expressions**, which are a concise way to create iterators.

Here's what it does:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

Let's break it down:

1. `{...}`: This is an expression that creates a set of values.
2. `for book in books`: This loops through the `books` collection, where `books` is likely a list or another iterable.
3. `if book.get("author")`: This filters out any items from `books` that don't have an "author" key.
4. `book.get("author")`: For each remaining item, this gets its author value (or `None` if the key doesn't exist).
5. `{...}`: The resulting values are stored in a set.

Now, let's talk about the magic keyword:

```python
yield from ...
```

This is called the **yield from** statement. It takes an iterable and "yields" each item from it, one at a time, without creating a new list or other data structure. This allows